In [1]:
from langchain.text_splitter import MarkdownTextSplitter, RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import dotenv
dotenv.load_dotenv()
FAQ_CHROMA_PATH="chroma_data"

#load myiam
with open("documents/myiam.txt", "r") as f:
    text = f.read()

# Chunk the text
splitter = MarkdownTextSplitter(chunk_size=1500, chunk_overlap=50)
chunks = splitter.split_text(text)

# Merge headers with content
documents = []
for chunk in chunks: 
    header = chunk.split('\n')[0]  # Assuming header is followed by /n
    documents.append(Document(page_content=chunk, metadata={"header": header, "category": "MyIAM"}))


In [2]:
# load PO
with open("documents/product_owner.txt", "r") as f:
    text = f.read()

# Chunk the text
splitter = MarkdownTextSplitter(chunk_size=1024, chunk_overlap=50)
chunks = splitter.split_text(text)

# Merge headers with content
for chunk in chunks:  
    header = chunk.split('\n')[0]
    documents.append(Document(page_content=chunk, metadata={"header": header, "category": "PO"}))

Huggingfaceembeddings downloads the model the first time and then it uses it locally, but we want it in local from the get to go. You need sentence-transformers pip installed to use them.

For getting the model locally you can go to 
- [https://huggingface.co/sentence-transformers/all-mpnet-base-v2](https://huggingface.co/sentence-transformers/all-mpnet-base-v2)  Better quality but slower: 1m 32 seconds
- [https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2)  Fast and lightweight: 4 segs

Click "Files and versions" tab
Download all the files to a local directory WITH GIT LFS 
```shell
./models/all-MiniLM-L6-v2/
├── config.json
├── pytorch_model.bin
├── tokenizer.json
├── tokenizer_config.json
├── vocab.txt
└── modules.json
```
### with sentence-transformers directly

In [3]:
from langchain_chroma import Chroma
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("./models/all-MiniLM-L6-v2")

# Create wrapper for LangChain
class LocalEmbeddings:
    def __init__(self, model):
        self.model = model
    
    def embed_documents(self, texts):
        return self.model.encode(texts, convert_to_tensor=False).tolist()
    
    def embed_query(self, text):
        return self.model.encode([text], convert_to_tensor=False)[0].tolist()

embeddings = LocalEmbeddings(model)

# Create vector store
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory=FAQ_CHROMA_PATH
)

### with HuggingFaceEmbeddings

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
# MODEL_NAME="./models/all-MiniLM-L6-v2"
MODEL_NAME="./models/all-mpnet-base-v2"

# Initialize local embeddings
embeddings = HuggingFaceEmbeddings(
    model_name=MODEL_NAME, 
    model_kwargs={'device': 'cpu'}  # or 'cuda' if you have GPU 
)

# Create vector store
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory=FAQ_CHROMA_PATH
)

Lets try some for size

In [7]:
vector_store_db= Chroma(
    persist_directory=FAQ_CHROMA_PATH,
    embedding_function=embeddings,
)

# Query the vector store
query = "What is ROBOTIC responsability?"
docs = vector_store_db.similarity_search(query, k=3)
print(docs[0].page_content) 

# Products and subscriptions
ROBOTIC team is responsible for designing and managing DPI (the product library and action broker) and VANISH (the orchestrator).
The PO uses these tools to define and manage their own products, as varied as Linux VM with an Apache server installed, an AVI, or an extra Linux Administrative account. A subscription is an instance of a product requested and owned by an user. It is created, destroyed and managed by its requester using the Vanish flows defined by the PO. Subscriptions are paused during each action execution to prevent concurrent actions from being launched.


In [8]:
query = "what are cyberark entitlements?"
docs = vector_store_db.similarity_search(query, k=3)
print(docs[0].page_content) 

### 1.2 Cyberark entitlements
Onboarded in ELDAP groups in EMEA and APAC and in AD in AMER, this entitlements grant access to servers or applications through Cyberark site. For more information, please follow the cyberark training (link at the bottom)
Example:
Entitlement: IV2 EMEA Cyberark ROBOTIC-TOOLS Breakglass
ELDAP group or AD group (only for AMER Cyberark entitlements): cn=IV2EMEA_CYBERARK_IV2AROBOTTOOL_Bglass,ou=group,ou=CyberArk,ou=Applications,dc=root
For a detailed explanation of what exact access gives each kind of entitlement (breakglass, user, dev & owner) please refer to the extended rights documentation (https://collab.cib.echonet/iandp/sites/Risk-and-Strategy/IT%20Security%20Operations/SecProd-OS-and-Services/UserGuides/SitePages/Home.aspx )
- For Cyberark entitlements ROBOTIC can onboard only human users (UIDs) which account has been activated at least 12 hours prior
- ROBOTIC cannot onboard here any non human account (SVC, GEN, OPS, …). The SVC and GEN accounts for m

In [9]:
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_chroma import Chroma
from langchain.schema.runnable import RunnablePassthrough
from langchain_huggingface import HuggingFaceEmbeddings

import dotenv
dotenv.load_dotenv()
FAQ_CHROMA_PATH="chroma_data"
MODEL_NAME="./models/all-mpnet-base-v2"

# First define the system & human prompts -> chat prompt template
roboasistant_system_template_str = """You are a member of the robotic team. 
Your job is to answer questions about the onboarding of entitlements or the product ownership.
Use the following context to answer questions.
Be as detailed as possible, but don't make up any information
that's not from the context. If you don't know an answer, say
you don't know.
{context}
"""

roboasistant_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=roboasistant_system_template_str
    )
)

roboasistant_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["question"], template="{question}")
)
roboassistant_messages = [roboasistant_system_prompt, roboasistant_human_prompt]

roboassistant_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"], messages=roboassistant_messages
)


# Second define the RAG chain = retriever, chat prompt template, chat model & output parser
chat_model = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

output_parser = StrOutputParser()

embeddings  = HuggingFaceEmbeddings(
    model_name=MODEL_NAME, #local path
    model_kwargs={'device': 'cpu'}  # or 'cuda' if you have GPU 
)

faqs_vector_db = Chroma(
    persist_directory=FAQ_CHROMA_PATH,
    embedding_function=embeddings,
)

faqs_retriever = faqs_vector_db.as_retriever(k=3)

faqs_roboassistant_chain = (
    {"context": faqs_retriever, "question": RunnablePassthrough()}
    | roboassistant_prompt_template
    | chat_model
    | StrOutputParser()
)

In [11]:
faqs_roboassistant_chain.invoke("what is the name of robotic in myiam?")

'In MyIAM, the ROBOTIC team is known as IV2 Manual Provisioner.'

In [12]:
faqs_roboassistant_chain.invoke("what are the cyberark entitlements?")

'Cyberark entitlements are permissions granted to users to access servers or applications through the Cyberark site. These entitlements are onboarded in ELDAP groups in EMEA and APAC and in AD in AMER. Examples of Cyberark entitlements include "IV2 EMEA Cyberark ROBOTIC-TOOLS Breakglass." Each entitlement is associated with a specific ELDAP group or AD group for AMER Cyberark entitlements. It\'s important to note that only human users (UIDs) with activated accounts at least 12 hours prior can be onboarded for Cyberark entitlements. Additionally, certain restrictions apply, such as not onboarding non-human accounts (SVC, GEN, OPS) and not onboarding the Breakglass entitlement to a user who already has the User entitlement. For more detailed information on the access provided by each type of entitlement, users can refer to the extended rights documentation.'